*   Предсказывать будем значение Price (задача регрессии).
*   Метрика: R2_score
*   Кросс-валидация на 10 частей с перемешиванием выборки, random_state=0

Ссылка на датасет: https://www.kaggle.com/vishnuvarthanrao/windows-store

In [ ]:
import pandas as pd # для фреймов данных
import numpy as np # для массивов и математических функций

%matplotlib inline
import matplotlib.pyplot as plt # для построения

import warnings
warnings.filterwarnings('ignore') # игнор любых предупреждений

In [ ]:
microsoft_df = pd.read_csv('/content/drive/MyDrive/University/Machine Learning/Project/msft.csv')

In [ ]:
# избавился от пустых строк

microsoft_df = microsoft_df.dropna(axis='index', how='any')
microsoft_df

,Name,Rating,No of people Rated,Category,Date,Price
0,Dynamic Reader,3.5,268,Books,07-01-2014,Free
1,"Chemistry, Organic Chemistry and Biochemistry-...",3.0,627,Books,08-01-2014,Free
2,BookViewer,3.5,593,Books,29-02-2016,Free
3,Brick Instructions,3.5,684,Books,30-01-2018,Free
4,Introduction to Python Programming by GoLearni...,2.0,634,Books,30-01-2018,Free
...,...,...,...,...,...,...
5316,Get Color,3.0,785,Developer Tools,08-08-2019,₹ 54.50
5317,JS King,1.0,720,Developer Tools,19-07-2018,₹ 269.00
5318,MQTTSniffer,2.5,500,Developer Tools,10-04-2017,₹ 64.00
5319,"Dev Utils - JSON, CSV and XML",4.0,862,Developer Tools,18-11-2019,₹ 269.00


In [ ]:
#@title Текст заголовка по умолчанию
microsoft_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5321 entries, 0 to 5320
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                5321 non-null   object 
 1   Rating              5321 non-null   float64
 2   No of people Rated  5321 non-null   int64  
 3   Category            5321 non-null   object 
 4   Date                5321 non-null   object 
 5   Price               5321 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 291.0+ KB


In [ ]:
microsoft_df['Price'].describe()

count     5321
unique      44
top       Free
freq      5163
Name: Price, dtype: object

In [ ]:
microsoft_df.sort_values(by='Price',ascending=False)[16:] # удаление первых 16 строк

,Name,Rating,No of people Rated,Category,Date,Price
5292,Javascript Studio Pro,4.0,384,Developer Tools,25-07-2015,₹ 624.00
5184,Hubble Space Telescope,4.5,866,Books,27-08-2013,₹ 59.00
5197,Clean Reader,4.5,768,Books,16-09-2019,₹ 59.00
5233,PDF Reader for Adobe PDF,3.0,354,Business,31-12-2019,₹ 574.00
5257,Universal Email App - Mail for All Mailbox,1.5,855,Business,27-02-2019,₹ 549.00
...,...,...,...,...,...,...
1773,Repost for Windows 10,4.0,740,Social,02-04-2018,Free
1772,Messenger For UWP,3.0,333,Social,06-08-2019,Free
1771,Talk to Someone,4.0,494,Social,27-12-2012,Free
1770,GMessenger Free,4.0,678,Social,11-04-2017,Free


In [ ]:
microsoft_df.sort_values(by='Price',ascending=True)[:5]

,Name,Rating,No of people Rated,Category,Date,Price
0,Dynamic Reader,3.5,268,Books,07-01-2014,Free
3447,Guns & Ammo,2.5,369,Lifestyle,20-02-2015,Free
3446,Cinematime,4.0,929,Lifestyle,19-12-2013,Free
3445,Sathya Sai Inspires,5.0,520,Lifestyle,19-03-2013,Free
3444,Airsoft International,2.0,760,Lifestyle,10-03-2014,Free


In [ ]:
microsoft_df.sort_values(by='No of people Rated', ascending = True)[:5]

,Name,Rating,No of people Rated,Category,Date,Price
4805,iNi Reader,4.0,100,News and Weather,04-08-2014,Free
1295,Python 3.9 (Beta),5.0,100,Developer Tools,19-11-2019,Free
140,Deaf Bible,5.0,100,Books,04-08-2016,Free
3188,Shot Notes,4.5,100,Lifestyle,04-09-2017,Free
3190,Survey for money: earn money paid surveys guide,4.5,100,Lifestyle,21-11-2018,Free


In [ ]:
microsoft_df.sort_values(by='No of people Rated', ascending = False)[:5]

,Name,Rating,No of people Rated,Category,Date,Price
3940,Guitar Lessons Beginners #1,3.5,1000,Music,06-01-2013,Free
3260,Daily Panchang,3.5,1000,Lifestyle,20-01-2016,Free
560,CDBS Facility Query,3.0,1000,Books,06-12-2016,Free
1929,Psychic Reader Paranormal,3.5,1000,Social,02-10-2014,Free
4306,Vinyl Player,4.0,999,Music,07-11-2014,Free


In [ ]:
features_df = microsoft_df[['Rating','No of people Rated','Category','Date']]

In [ ]:
features_df = features_df.dropna() # удаление строк NaN

In [ ]:
features_df.describe()

,Rating,No of people Rated
count,5321.000000,5321.000000
mean,3.792708,551.798722
std,0.991681,259.787733
min,1.000000,100.000000
25%,3.500000,327.000000
50%,4.000000,548.000000
75%,4.500000,779.000000
max,5.000000,1000.000000


# Функциональная инженерия

In [ ]:
microsoft_df['Category'].value_counts()

Music                      753
Books                      707
Business                   679
Health and Fitness         527
Developer Tools            500
Lifestyle                  492
News and Weather           483
Kids and Family            345
Social                     328
Food and Dining            166
Navigation and Maps        154
Multimedia Design          117
Government and Politics     70
Name: Category, dtype: int64

In [ ]:
genre_list = ['Music', 'Books', 'Business', 'Health and Fitness', 'Developer Tools', 'Lifestyle', 'News and Weather', 
'Kids and Family', 'Social', 'Food and Dining', 'Navigation and Maps', 'Multimedia Design', 'Government and Politics']

In [ ]:
for genre in genre_list:
  features_df[genre] = features_df['Category'].apply(lambda x: 1 if genre in x else 0)

In [ ]:
features_df.describe()

,Rating,No of people Rated,Music,Books,Business,Health and Fitness,Developer Tools,Lifestyle,News and Weather,Kids and Family,Social,Food and Dining,Navigation and Maps,Multimedia Design,Government and Politics
count,5321.000000,5321.000000,5321.000000,5321.000000,5321.000000,5321.000000,5321.000000,5321.000000,5321.000000,5321.000000,5321.000000,5321.000000,5321.000000,5321.000000,5321.000000
mean,3.792708,551.798722,0.141515,0.132870,0.127608,0.099042,0.093967,0.092464,0.090772,0.064837,0.061643,0.031197,0.028942,0.021988,0.013155
std,0.991681,259.787733,0.348585,0.339466,0.333684,0.298746,0.291811,0.289707,0.287312,0.246262,0.240528,0.173867,0.167659,0.146659,0.113951
min,1.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.500000,327.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,548.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.500000,779.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,1000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# удаление стобца

features_df = features_df.drop(columns=['Category'])

In [ ]:
features_df.sort_values(by='Date',ascending=True)[:5]

,Rating,No of people Rated,Date,Music,Books,Business,Health and Fitness,Developer Tools,Lifestyle,News and Weather,Kids and Family,Social,Food and Dining,Navigation and Maps,Multimedia Design,Government and Politics
854,4.0,263,01-01-2014,0,0,1,0,0,0,0,0,0,0,0,0,0
5099,5.0,183,01-01-2015,0,0,0,0,0,0,1,0,0,0,0,0,0
2772,3.0,362,01-01-2015,0,0,0,1,0,0,0,0,0,0,0,0,0
3368,3.5,493,01-01-2015,0,0,0,0,0,1,0,0,0,0,0,0,0
4140,4.0,626,01-01-2015,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# преобразовать Date объект в строку

#features_Date = np.array(features_df['Date'])
from datetime import datetime

features_df = features_df.assign(Day = 0)
sdate=datetime(2010, 11, 4)
features_df['Day']=features_df['Date'].apply(lambda x: (datetime.strptime(x, '%d-%m-%Y')-sdate).days)
features_df = features_df.drop(columns='Date')
features_df.head(8)

,Rating,No of people Rated,Music,Books,Business,Health and Fitness,Developer Tools,Lifestyle,News and Weather,Kids and Family,Social,Food and Dining,Navigation and Maps,Multimedia Design,Government and Politics,Day
0,3.5,268,0,1,0,0,0,0,0,0,0,0,0,0,0,1160
1,3.0,627,0,1,0,0,0,0,0,0,0,0,0,0,0,1161
2,3.5,593,0,1,0,0,0,0,0,0,0,0,0,0,0,1943
3,3.5,684,0,1,0,0,0,0,0,0,0,0,0,0,0,2644
4,2.0,634,0,1,0,0,0,0,0,0,0,0,0,0,0,2644
5,4.5,399,0,1,0,0,0,0,0,0,0,0,0,0,0,2267
6,4.5,566,0,1,0,0,0,0,0,0,0,0,0,0,0,740
7,3.0,175,0,1,0,0,0,0,0,0,0,0,0,0,0,1078


# Работа с целевым признаком

In [ ]:
#исправление string в float
microsoft_df.fillna(0.0, inplace=True)

microsoft_df = microsoft_df.assign(**{
    'Price' : microsoft_df['Price'].str.replace('Free', '0')})

microsoft_df = microsoft_df.assign(**{
    'Price' : microsoft_df['Price'].str.replace('₹ ', '')})

microsoft_df = microsoft_df.assign(**{
    'Price' : microsoft_df['Price'].str.replace('\\...', '')})

microsoft_df = microsoft_df.assign(**{
    'Price' : microsoft_df['Price'].str.replace(',', '.').astype(float)})

In [ ]:
labels_df = microsoft_df['Price']

In [ ]:
labels_df = labels_df[features_df.index]

In [ ]:
labels_df

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
        ...  
5316     54.0
5317    269.0
5318     64.0
5319    269.0
5320    219.0
Name: Price, Length: 5321, dtype: float64

# Регрессия

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=0)

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_predict

lasso = linear_model.Lasso()
linear_pred = cross_val_predict(lasso, features_df, labels_df, cv=kf)
print("Оценки с перекрестной проверкой:", linear_pred)

Оценки с перекрестной проверкой: [ 1.23363359  4.59934158  7.12354842 ... 20.26726134 18.10621303
 -4.73479803]


In [ ]:
# чем ближе к 1 - тем лучше

from sklearn.metrics import r2_score
print('Результат линейной регрессии:', r2_score(labels_df, linear_pred))

Результат линейной регрессии: 0.07559803717489533


In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

extraTrees = ExtraTreesRegressor()
tress_pred = cross_val_predict(extraTrees, features_df, labels_df, cv=kf)
print('Результат регрессии дополнительных деревьев:', r2_score(labels_df, tress_pred))

Результат регрессии дополнительных деревьев: 0.2001498673739729


In [ ]:
from sklearn.ensemble import RandomForestRegressor

randomForest = RandomForestRegressor()
forest_pred = cross_val_predict(randomForest, features_df, labels_df, cv=kf)
print('Результат регрессии случайный лес:', r2_score(labels_df, forest_pred))

Результат регрессии случайный лес: 0.27038153178863045


In [ ]:
from sklearn.ensemble import AdaBoostRegressor

AdaBoost = AdaBoostRegressor()
ada_pred = cross_val_predict(AdaBoost, features_df, labels_df, cv=kf)
print('Результат регрессии Ада Бустинг:', r2_score(labels_df, ada_pred))

Результат регрессии Ада Бустинг: 0.08443313613583159


In [ ]:
from sklearn.ensemble import BaggingRegressor

Bagging = BaggingRegressor()
bagging_pred = cross_val_predict(Bagging, features_df, labels_df, cv=kf)
print('Результат регрессии "Упаковка":', r2_score(labels_df, bagging_pred))

Результат регрессии "Упаковка": 0.22018760101331203


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

GradientBoosting = GradientBoostingRegressor()
gradient_pred = cross_val_predict(GradientBoosting, features_df, labels_df, cv=kf)
print('Результат регрессии повышение градиента:', r2_score(labels_df, gradient_pred))

Результат регрессии повышение градиента: 0.35599710815773966


In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge_pred = cross_val_predict(ridge, features_df, labels_df, cv=kf)
print('Результат регрессии Ridge:', r2_score(labels_df, ridge_pred))

Результат регрессии Ridge: 0.09681050656257484


In [ ]:
from sklearn.linear_model import TheilSenRegressor

theilsen = TheilSenRegressor()
theilsen_pred = cross_val_predict(theilsen, features_df, labels_df, cv=kf)
print('Результат регрессии Тейл Сен:', r2_score(labels_df, theilsen_pred))

Результат регрессии Тейл Сен: -0.01949998752683224


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

kneighbors = KNeighborsRegressor()
kneighbors_pred = cross_val_predict(kneighbors, features_df, labels_df, cv=kf)
print('Результат регрессии KNeighbors:', r2_score(labels_df, kneighbors_pred))

Результат регрессии KNeighbors: 0.10602527442974441


In [ ]:
corr_matrix = pd.concat([features_df, labels_df],axis=1).corr()

corr_matrix['Price'].sort_values(ascending=False)

Price                      1.000000
Day                        0.166256
Business                   0.130576
Developer Tools            0.099801
Books                      0.051447
No of people Rated        -0.000341
Government and Politics   -0.016124
Multimedia Design         -0.020939
Navigation and Maps       -0.024109
Food and Dining           -0.025059
Social                    -0.035792
Kids and Family           -0.036771
News and Weather          -0.044124
Lifestyle                 -0.044574
Health and Fitness        -0.046301
Music                     -0.056698
Rating                    -0.208100
Name: Price, dtype: float64

In [ ]:
from xgboost import XGBRegressor

XGBR = XGBRegressor()
xgbr_pred = cross_val_predict(XGBR, features_df, labels_df, cv=kf)
print('Результат регрессии XGBR:', r2_score(labels_df, xgbr_pred))

[06:39:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:39:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:39:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:39:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:39:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:39:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:39:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:39:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

# Нейронка

In [ ]:
#import tensorflow as tf
#import tensorflow.keras.layers as L
#from keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
#def get_tf_model():
#    model = tf.keras.Sequential([
#        L.Input(shape=(features_df.shape[1])),
#        L.Dense(100, activation='relu'),
#        L.BatchNormalization(),
#        L.Dense(100, activation='relu'),
#        L.BatchNormalization(),
#        L.Dense(100, activation='relu'),
#        L.BatchNormalization(),
#        L.Dense(1)
#    ])

#    model.compile(
#        optimizer='adam',
#        loss = 'mse',
#        metrics=['accuracy','mse']
#    )
    
#    return model

In [ ]:
#get_tf_model().summary()

In [ ]:
#keras = KerasRegressor(build_fn = get_tf_model, epochs = 10, verbose = 0, batch_size = 100)
#keras_pred = cross_val_predict(keras, features_df, labels_df, cv=kf)
#print('Результат регрессии Ridge:', r2_score(labels_df, keras_pred))

# Улучшение

In [ ]:
# попробуем улучшить некоторые из этих результатов
# поиск по сетке показывает, какие комбинации гиперпараметров обеспечивают наилучшие результаты, 
# пробуя множество различных комбинаций

from sklearn.model_selection import GridSearchCV #RandomSearch

# Введите модель, параметры настроек и возвращение лучшуей модели
def grid_search(model,params):
  search = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error')
  search.fit(features_df, labels_df)
  return search.best_estimator_

In [ ]:
XGBR.get_params()

In [ ]:
# Выберем несколько вариантов и применим поиск по сетке

import warnings
warnings.filterwarnings('ignore') # игнор любых предупреждений

param_grid = [
              {'alpha': [0.1,0.05,0.01,0.005] , #либо диапозон (0.1 - 0.005)
               "fit_intercept": [True, False], 
               'normalize': [True, False],
               "tol": [0.0005,.0001,0.00005]}
]

#param_grid = { 'objective':'reg:squarederror',
#         'max_depth': 6, 
#         'colsample_bylevel':0.5,
#        'learning_rate':0.01,
#         'random_state':20}


xgbr_grid = grid_search(XGBR, param_grid)

xgbr_grid.get_params() # это будут наши новые параметры

In [ ]:
XGBR = XGBRegressor(
 alpha = 0.5, # либо подбор руками подобных значений тут
 copy_X = True,
 fit_intercept = True, 
 max_iter = 10,
 normalize = True,
 positive = False,
 precompute = False,
 random_state = 0,
 selection = 'cyclic',
 tol = 0.00005,
 warm_start = False)
xgbr_pred = cross_val_predict(XGBR, features_df, labels_df, cv=kf)
print('Результат улучшенной регрессии:', r2_score(labels_df, xgbr_pred))